In [7]:
import pandas as pd
from copy import deepcopy as dc
import brightway2 as bw

In [2]:
def load_data(data_path, NW_path):
    sheet_names = pd.read_excel(data_path, sheet_name=None)
    sheet_names = [sheet for sheet in sheet_names.keys()]

    df = pd.read_excel(data_path, sheet_name=sheet_names[0])
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
    
    data_NW = pd.read_excel(NW_path)
    norm_lst = data_NW['Normalization'].tolist()
    weigh_lst = data_NW['Weighting'].tolist()

    return df, norm_lst, weigh_lst

In [3]:
# Function to obtain the LCIA category to calculate the LCIA results
def lcia_method(method):
    # Checking if the LCIA method is ReCiPe, and ignores difference between lower and upper case 
    if 'recipe' in method.lower():
        # Using H (hierachly) due to it has a 100 year span
        # Obtaining the midpoint categpries and ignoring land transformation (Land use still included)
        all_methods = [m for m in bw.methods if 'ReCiPe 2016 v1.03, midpoint (H)' in str(m) and 'no LT' not in str(m)] # Midpoint

        # Obtaining the endpoint categories and ignoring land transformation
        endpoint = [m for m in bw.methods if 'ReCiPe 2016 v1.03, endpoint (H)' in str(m) and 'no LT' not in str(m) and 'total' in str(m)]

        # Combining midpoint and endpoint, where endpoint is added to the list of the midpoint categories
        for meth in endpoint:
            all_methods.append(meth)

        print('Recipe is selected')

    # Checking if EF is choses for the LCIA method
    elif 'ef' in method.lower():
        all_methods = [m for m in bw.methods if 'EF v3.1 EN15804' in str(m) and "climate change:" not in str(m)]
        print('EF is selected')

    else:
        print('Select either EF or ReCiPe as the LCIA methos')
        all_methods = []

    # Returning the selected LCIA methods
    return all_methods

In [ ]:
# Function to obtaining where the LCIA results start
def obtaining_lcia_start_index(df):
    idx = 0
    for i, col in enumerate(df.columns):
        if 'acidification' in col:
            idx = i
            break
    return idx

In [ ]:
def obtaining_norm_weight_single_score_results(data_path, NW_path):
        
    df, norm_lst, weigh_lst = load_data(data_path, NW_path)

    weigh_df = dc(df)

    idx = obtaining_lcia_start_index(df)
    data_cols = df.columns[:idx].to_list()

    # Obtaining the impact categories from EF
    ic = lcia_method('ef')
    ic = [str(i).replace(f' EN15804', '') for i in ic ]

    new_cols = data_cols + ic
    df_new = pd.DataFrame(0, index=df.index,columns=new_cols, dtype=object)

    # Setting the values in the dataframe for which will be normalized, weighted and turned into a single score
    for col in df_new.columns:
        col_str = str(col)
        for idx, row in df_new.iterrows():
            row[col] = df.at[idx, col_str]   

        
    weigh_df = dc(df_new)
    norm_df = dc(df_new)

    # Creating the normalized data
    for col_idx, col in enumerate(ic):
        for row_idx, row in norm_df.iterrows():
            row[col] =  df_new.at[row_idx, col] * norm_lst[col_idx]

    # Creating the weighted data
    for col_idx, col in enumerate(ic):
        for row_idx, row in weigh_df.iterrows():
            row[col] =  norm_df.at[row_idx, col] * weigh_lst[col_idx]

    # Adding an extra column called single score 
    sc_col = data_cols
    if sc_col[-1] != 'single score':
        sc_col.append('single score')


    single_score_df = pd.DataFrame(0, index=df.index, columns=sc_col, dtype=object)

    # creating the dataframe to calculate the single score from
    for col in df_new.columns:
        col_str = str(col)
        for idx, row in single_score_df.iterrows():
            try:
                row[col] = df.at[idx, col_str] 
            except ValueError:
                pass
    # Calculating the single score values
    for idx_val, row in weigh_df.iterrows():
        temp = 0
        for i in ic:
            temp += row[i]
        single_score_df.at[idx_val, 'single score'] = temp

    # Saving all the results to the same sheet
    writer = pd.ExcelWriter(data_path, engine = 'xlsxwriter')
    df.to_excel(writer, sheet_name = 'data')
    norm_df.to_excel(writer, sheet_name = 'normalized')
    weigh_df.to_excel(writer, sheet_name = 'weighted')
    single_score_df.to_excel(writer, sheet_name = 'single score')
    writer.close()

In [6]:

data_path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Brighway\all comparison.xlsx'
NW_path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Norm + Weigh.xlsx'


obtaining_norm_weight_single_score_results(data_path, NW_path)


EF is selected
